In [ ]:
# --- Step 1: Install Required Libraries ---
!pip install python-docx
!pip install pymongo[srv]

# --- Step 2: Mount Google Drive ---
from google.colab import drive
drive.mount('/content/drive')


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 34.3 MB/s eta 0:00:00
Mounted at /content/drive


In [ ]:
# --- Step 3: Import Libraries ---
import os
import re
import time
from docx import Document
from pymongo import MongoClient
from tqdm import tqdm

In [ ]:
from google.colab import files
uploaded = files.upload()  # Upload your service account JSON here

Saving ai-alumni-chatbot-36a8abc75502.json to ai-alumni-chatbot-36a8abc75502.json


In [ ]:
import firebase_admin
from firebase_admin import credentials, firestore

# Replace 'your-service-account.json' with your uploaded file name
cred = credentials.Certificate('ai-alumni-chatbot-36a8abc75502.json')
firebase_admin.initialize_app(cred)

db = firestore.client()



In [ ]:


# Sample alumni profile for testing
alumni_profile = {
    "Name": "John Carter",
    "Email": "jeffery26@example.org",
    "Phone": "(864)217-2320",
    "Location": "Port Christopher, Mexico",
    "Skills": ["Java", "Python", "Vue", "PostgreSQL", "Google Cloud"],
    "WorkExperience": ["Cybersecurity Engineer at Delacruz, Norris and Allen", "Cybersecurity Engineer at Campbell, Price and Wise"],
    "Education": ["BSc in Computer Science - Lyons-Smith", "BSc in Computer Science - Robinson Inc"],
    "source_resume": "John_Carter.docx"
}

# Insert this test profile into Firestore
doc_ref = db.collection('alumni_profiles').document()
doc_ref.set(alumni_profile)

print("✅ Test alumni profile inserted into Firestore!")

doc_ref = db.collection('alumni_profiles').document()
doc_ref.set(alumni_profile)


✅ Test alumni profile inserted into Firestore!


update_time {
  seconds: 1745748747
  nanos: 716941000
}

In [ ]:
# --- Extract Resume Text ---
def extract_text_from_docx(file_path):
    doc = Document(file_path)
    full_text = [para.text.strip() for para in doc.paragraphs if para.text.strip()]
    return full_text

# --- Parse Resume to Profile ---
def parse_resume(resume_text, source_filename=None):
    alumni_profile = {}
    alumni_profile["Name"] = resume_text[0] if len(resume_text) > 0 else None

    if len(resume_text) > 1:
        contact_info = resume_text[1]
        email_match = re.search(r"[\w\.-]+@[\w\.-]+", contact_info)
        phone_match = re.search(r"\(?\d{3}\)?[-.\s]?\d{3}[-.\s]?\d{4}", contact_info)
        location = contact_info.split('Location:')[-1].strip() if 'Location:' in contact_info else ""

        alumni_profile["Email"] = email_match.group() if email_match else None
        alumni_profile["Phone"] = phone_match.group() if phone_match else None
        alumni_profile["Location"] = location
    else:
        alumni_profile["Email"] = alumni_profile["Phone"] = alumni_profile["Location"] = None

    section_indices = {}
    for idx, line in enumerate(resume_text):
        if line.lower() in ["professional summary", "skills", "work experience", "education", "projects"]:
            section_indices[line.lower()] = idx

    skills = []
    if "skills" in section_indices and "work experience" in section_indices:
        skills = resume_text[section_indices["skills"]+1:section_indices["work experience"]]

    work_experience = []
    if "work experience" in section_indices and "education" in section_indices:
        work_experience = resume_text[section_indices["work experience"]+1:section_indices["education"]]

    education = []
    if "education" in section_indices and "projects" in section_indices:
        education = resume_text[section_indices["education"]+1:section_indices["projects"]]

    alumni_profile["Skills"] = skills
    alumni_profile["WorkExperience"] = work_experience
    alumni_profile["Education"] = education
    alumni_profile["source_resume"] = source_filename

    return alumni_profile

# --- Process Resumes with Insert into Firestore ---
def process_resumes_firestore(folder_path, batch_size=100):
    resumes = [f for f in os.listdir(folder_path) if f.endswith(".docx")]
    batch = []
    total_inserted = 0

    for idx, filename in enumerate(tqdm(resumes, desc="Processing Resumes")):
        try:
            file_path = os.path.join(folder_path, filename)
            resume_text = extract_text_from_docx(file_path)
            alumni_profile = parse_resume(resume_text, source_filename=filename)
            batch.append(alumni_profile)

            # Insert after reaching batch size
            if len(batch) >= batch_size:
                retry_count = 0
                while retry_count < 3:
                    try:
                        for profile in batch:
                            doc_ref = db.collection('alumni_profiles').document()
                            doc_ref.set(profile)
                        total_inserted += len(batch)
                        print(f"✅ Inserted batch of {len(batch)} resumes (Total inserted: {total_inserted})")
                        batch = []  # Reset batch
                        time.sleep(1)  # polite pause after each batch
                        break
                    except Exception as e:
                        retry_count += 1
                        print(f"⚠️ Retry {retry_count}/3 for batch due to: {e}")
                        time.sleep(2)
                if retry_count == 3:
                    print(f"❌ Failed to insert batch after 3 retries. Skipping...")
                    batch = []

        except Exception as e:
            print(f"⚠️ Error processing {filename}: {e}")

    # Insert any leftover resumes
    if batch:
        try:
            for profile in batch:
                doc_ref = db.collection('alumni_profiles').document()
                doc_ref.set(profile)
            total_inserted += len(batch)
            print(f"✅ Inserted final batch of {len(batch)} resumes (Total inserted: {total_inserted})")
        except Exception as e:
            print(f"⚠️ Final batch insert failed: {e}")

    print(f"\n🎯 Total {total_inserted} resumes inserted successfully!")

# --- Main Execution ---
resume_folder = "/content/drive/MyDrive/generated_resumes/"  # <-- your folder path
process_resumes_firestore(resume_folder, batch_size=100)


Processing Resumes:   3%|▎         | 98/3541 [00:49<02:31, 22.72it/s]

✅ Inserted batch of 100 resumes (Total inserted: 100)


Processing Resumes:   6%|▌         | 195/3541 [01:19<01:32, 36.00it/s]

✅ Inserted batch of 100 resumes (Total inserted: 200)


Processing Resumes:   8%|▊         | 296/3541 [01:49<01:26, 37.32it/s]

...

✅ Inserted batch of 100 resumes (Total inserted: 3400)


Processing Resumes:  99%|█████████▉| 3499/3541 [15:49<00:01, 35.90it/s]

✅ Inserted batch of 100 resumes (Total inserted: 3500)


Processing Resumes: 100%|██████████| 3541/3541 [16:02<00:00,  3.68it/s]


✅ Inserted final batch of 41 resumes (Total inserted: 3541)

🎯 Total 3541 resumes inserted successfully!
